必要なライブラリのインポート

In [4]:
import numpy as np
import pandas as pd
import faiss
from sentence_transformers import SentenceTransformer
from openai import OpenAI
from typing import List, Tuple

In [5]:
from pathlib import Path

# ===== 検索設定 =====
TOP_K = 5
MAX_CONTEXT_CHARS = 4000

# ===== （任意）評価データ設定 =====
EVAL_CSV = "コマンド推論用_1000_chunked.csv"
N_EVAL = 30



INDEX_ROOT = Path("index")

model_dirs = [
    d for d in INDEX_ROOT.iterdir()
    if d.is_dir()
    and (d / "embeddings.npy").exists()
    and (d / "chunks.parquet").exists()
]

print(f"モデル数: {len(model_dirs)}\n")

for d in model_dirs:
    embs = np.load(d / "embeddings.npy")
    chunks = pd.read_parquet(d / "chunks.parquet")

    print(f"【{d.name}】")
    print(f"  Embeddings shape: {embs.shape}")
    print(f"  Chunks: {len(chunks)} 件")
    print(f"  Columns: {chunks.columns.tolist()}")
    print("-" * 60)


モデル数: 5

【text-embedding-granite-embedding-107m-multilingual】
  Embeddings shape: (12239, 384)
  Chunks: 12239 件
  Columns: ['doc_idx', 'chunk_id', 'chunk_text', 'id']
------------------------------------------------------------
【text-embedding-embeddinggemma-300m】
  Embeddings shape: (12239, 768)
  Chunks: 12239 件
  Columns: ['doc_idx', 'chunk_id', 'chunk_text', 'id']
------------------------------------------------------------
【text-embedding-mxbai-embed-large-v1】
  Embeddings shape: (12239, 1024)
  Chunks: 12239 件
  Columns: ['doc_idx', 'chunk_id', 'chunk_text', 'id']
------------------------------------------------------------
【text-embedding-embeddinggemma-300m-qat】
  Embeddings shape: (12239, 768)
  Chunks: 12239 件
  Columns: ['doc_idx', 'chunk_id', 'chunk_text', 'id']
------------------------------------------------------------
【text-embedding-qwen3-embedding-0.6b】
  Embeddings shape: (12239, 1024)
  Chunks: 12239 件
  Columns: ['doc_idx', 'chunk_id', 'chunk_text', 'id']
--------

In [6]:
#エンコーダー初期化
print("LM Studio (OpenAI互換) クライアント初期化中...")

LM_STUDIO_BASE_URL = "http://localhost:1234/v1"
LM_STUDIO_API_KEY = "lm-studio"  # 何でもOK（LM Studioは基本見ない）

lm_client = OpenAI(
    base_url=LM_STUDIO_BASE_URL,
    api_key=LM_STUDIO_API_KEY
)

print("完了")
print("LM Studio:", LM_STUDIO_BASE_URL)

def lm_embed(texts, model_name: str):
    resp = lm_client.embeddings.create(
        model=model_name,
        input=texts
    )
    return np.array([d.embedding for d in resp.data], dtype=np.float32)

print("疎通確認中...")
v = lm_embed(["テスト"], model_dirs[0].name)  # model_dirs は index/ の各モデルdir
print("OK:", model_dirs[0].name, "dim=", v.shape[1])

for d in model_dirs:
    print("疎通確認:", d.name)
    try:
        v = lm_embed(["テスト"], d.name)
        print("  OK dim =", v.shape[1])
    except Exception as e:
        print("  NG:", e)


LM Studio (OpenAI互換) クライアント初期化中...
完了
LM Studio: http://localhost:1234/v1
疎通確認中...
OK: text-embedding-granite-embedding-107m-multilingual dim= 384
疎通確認: text-embedding-granite-embedding-107m-multilingual
  OK dim = 384
疎通確認: text-embedding-embeddinggemma-300m
  OK dim = 768
疎通確認: text-embedding-mxbai-embed-large-v1
  OK dim = 1024
疎通確認: text-embedding-embeddinggemma-300m-qat
  OK dim = 768
疎通確認: text-embedding-qwen3-embedding-0.6b
  OK dim = 1024


FAISSインデックス構築

In [7]:
def build_faiss_index(embeddings: np.ndarray) -> faiss.IndexFlatIP:
    """
    FAISSの内積インデックスを構築。
    L2正規化後の内積 = コサイン類似度
    """
    emb = embeddings.astype(np.float32).copy()
    faiss.normalize_L2(emb)

    d = emb.shape[1]
    index = faiss.IndexFlatIP(d)
    index.add(emb)
    return index

print("FAISSインデックスを構築中...")

faiss_indexes = {}

for d in model_dirs:
    embs = np.load(d / "embeddings.npy")
    idx = build_faiss_index(embs)
    faiss_indexes[d.name] = idx
    print(f"  {d.name}: {idx.ntotal} 件")



FAISSインデックスを構築中...
  text-embedding-granite-embedding-107m-multilingual: 12239 件
  text-embedding-embeddinggemma-300m: 12239 件
  text-embedding-mxbai-embed-large-v1: 12239 件
  text-embedding-embeddinggemma-300m-qat: 12239 件
  text-embedding-qwen3-embedding-0.6b: 12239 件


ベクトル検索関数

In [8]:
chunks_by_model = {}
text_col_by_model = {}
faiss_indexes = {}

def infer_text_col(df: pd.DataFrame) -> str:
    for c in ["chunk_text", "text", "content", "body", "page_content"]:
        if c in df.columns:
            return c
    obj_cols = [c for c in df.columns if df[c].dtype == "object"]
    return obj_cols[0] if obj_cols else df.columns[0]

for d in model_dirs:
    chunks = pd.read_parquet(d / "chunks.parquet")
    embs = np.load(d / "embeddings.npy").astype(np.float32)

    faiss.normalize_L2(embs)
    idx = faiss.IndexFlatIP(embs.shape[1])
    idx.add(embs)

    chunks_by_model[d.name] = chunks
    text_col_by_model[d.name] = infer_text_col(chunks)
    faiss_indexes[d.name] = idx

print("ready:", len(faiss_indexes), "models")


def search_model(query: str, model_name: str, top_k: int = TOP_K) -> pd.DataFrame:
    q_vec = lm_embed([query], model_name).astype(np.float32)
    faiss.normalize_L2(q_vec)

    index = faiss_indexes[model_name]
    scores, indices = index.search(q_vec, top_k)
    scores, indices = scores[0], indices[0]

    chunks = chunks_by_model[model_name]
    results = []
    for rank, (idx, score) in enumerate(zip(indices, scores), 1):
        if idx < 0:
            continue
        row = chunks.iloc[int(idx)].to_dict()
        row["rank"] = rank
        row["score"] = float(score)
        results.append(row)

    return pd.DataFrame(results)


ready: 5 models


LLM回答生成（OpenAI ChatGPT）

In [9]:
# ===== OpenAI（本家）設定 =====
from openai import OpenAI

LLM_MODEL = "gpt-4o"   # 回答生成用モデル
openai_client = OpenAI()

print("OpenAI client ready:", LLM_MODEL)


OpenAI client ready: gpt-4o


In [10]:
def llm_chat(messages: List[dict], max_tokens: int = 1024, temperature: float = 0.2) -> str:
    """
    OpenAI ChatGPT APIで回答生成
    """
    response = openai_client.chat.completions.create(
        model=LLM_MODEL,
        messages=messages,
        max_tokens=max_tokens,
        temperature=temperature,
    )
    return response.choices[0].message.content


def build_context(results: pd.DataFrame, max_chars: int = MAX_CONTEXT_CHARS) -> str:
    """
    検索結果からLLMに渡すコンテキストを構築
    """
    if len(results) == 0:
        return "(関連するコンテキストが見つかりませんでした)"
    
    parts = []
    total_len = 0
    
    # chunk_text列の名前を確認
    text_col = "chunk_text" if "chunk_text" in results.columns else "zenbun"
    
    for _, row in results.iterrows():
        # ヘッダー
        doc_idx = row.get("doc_idx", "?")
        chunk_id = row.get("chunk_id", "?")
        midashi = row.get("midashi_1", "")
        header = f"[doc_idx={doc_idx}, chunk_id={chunk_id}, title={midashi}]"
        
        # 本文
        body = str(row.get(text_col, ""))
        block = f"{header}\n{body}\n"
        
        if total_len + len(block) > max_chars:
            break
        
        parts.append(block)
        total_len += len(block)
    
    return "\n---\n".join(parts)


def generate_answer(question: str, context: str) -> str:
    """
    コンテキストを元にLLMで回答生成
    """
    system_prompt = """あなたは日本語で回答する原子力・工学系の専門アシスタントです。
以下の「コンテキスト」に含まれる情報を最優先して使い、
コンテキストに書いていないことは憶測で補わず、
その場合は「資料からは読み取れません」と答えてください。"""

    user_prompt = f"""# 質問
{question}

# コンテキスト（検索結果）
{context}
"""

    return llm_chat([
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ])

RAG統合関数

In [11]:
from typing import Tuple

def rag_with_model(
    question: str,
    model_name: str,
    top_k: int = TOP_K
) -> Tuple[str, pd.DataFrame]:
    """
    index 配下の embedding model を使った RAG
    """
    print(f"[{model_name}] 検索中...")
    results = search_model(question, model_name, top_k)

    if len(results) == 0:
        return "関連するコンテキストが見つかりませんでした。", results

    context = build_context(results)
    print(f"[{model_name}] 回答生成中...")
    answer = generate_answer(question, context)

    # 参照IDを末尾に追加（元コードそのまま）
    ref_ids = results["id"].unique().tolist() if "id" in results.columns else []
    if ref_ids:
        answer = answer.strip() + f"\n\n【参照資料ID】: {', '.join(map(str, ref_ids))}"

    return answer.strip(), results


In [12]:
def display_search_results(results: pd.DataFrame, top_k: int, model_name: str):
    """
    検索結果を詳細表示
    """
    if len(results) == 0:
        print("検索結果なし")
        return

    text_col = "chunk_text" if "chunk_text" in results.columns else "zenbun"

    for _, row in results.head(top_k).iterrows():
        print(f"\n--- Rank {row.get('rank', '?')} (Score: {row.get('score', 0):.4f}) ---")
        print(f"ID: {row.get('id', '?')}")
        print(f"見出し: {row.get('midashi_1', '?')}")
        print(f"開催日: {row.get('kaisai_date', '?')}")
        print("テキスト:")
        print(row.get(text_col, ""))


In [13]:
def compare_rag_all(question: str, top_k: int = TOP_K):
    """
    index 配下の全モデルでRAGを実行して比較
    """
    print("=" * 60)
    print(f"質問: {question}")
    print("=" * 60)

    out = {}

    for model_dir in model_dirs:
        model_name = model_dir.name

        print("\n" + "-" * 40)
        answer, results = rag_with_model(question, model_name, top_k)

        print(f"\n【{model_name} 回答】")
        print(answer)

        print(f"\n【検索結果（上位{top_k}件）】")
        display_search_results(results, top_k, model_name)

        out[model_name] = {"answer": answer, "results": results}

    return out


テスト実行

In [14]:
# LLM接続テスト（OpenAI ChatGPT）
print(f"LLM接続テスト（モデル: {LLM_MODEL}）...")
try:
    test_response = llm_chat([{"role": "user", "content": "接続テストです。OKと返してください。"}])
    print(f"応答: {test_response}")
    print("ChatGPT接続OK")
except Exception as e:
    print(f"ChatGPT接続エラー: {e}")

LLM接続テスト（モデル: gpt-4o）...
応答: OK
ChatGPT接続OK


In [15]:
# 検索テスト
test_query = "原子力発電所の安全対策について"

print("=== 検索テスト ===")
print(f"クエリ: {test_query}\n")

for model_dir in model_dirs:
    model_name = model_dir.name
    print(f"【{model_name}】")

    results = search_model(test_query, model_name, top_k=3)

    if len(results) > 0:
        # 表示したい列が無い場合に備えて安全に
        cols = [c for c in ["rank", "score", "doc_idx", "chunk_id"] if c in results.columns]
        print(results[cols].to_string(index=False))
    else:
        print("結果なし")

    print()


=== 検索テスト ===
クエリ: 原子力発電所の安全対策について

【text-embedding-granite-embedding-107m-multilingual】
 rank    score  doc_idx  chunk_id
    1 0.843693       54         0
    2 0.823897       85         0
    3 0.823058      665         0

【text-embedding-embeddinggemma-300m】
 rank    score  doc_idx  chunk_id
    1 0.612874      403         0
    2 0.612011      360         0
    3 0.608926      423         0

【text-embedding-mxbai-embed-large-v1】
 rank    score  doc_idx  chunk_id
    1 0.888892      263         0
    2 0.883632      218         0
    3 0.882227      256         0

【text-embedding-embeddinggemma-300m-qat】
 rank    score  doc_idx  chunk_id
    1 0.649673      565         0
    2 0.645446      360         0
    3 0.640284      403         0

【text-embedding-qwen3-embedding-0.6b】
 rank    score  doc_idx  chunk_id
    1 0.746069      287         0
    2 0.741891      247         0
    3 0.741627      284         0



使用例

In [16]:
# 質問を入力してRAG実行
question = """
伊方発電所3号機の安全対策について教えてください。
"""

# 両モデルで比較
comparison = compare_rag_all(question.strip(), top_k=10)

質問: 伊方発電所3号機の安全対策について教えてください。

----------------------------------------
[text-embedding-granite-embedding-107m-multilingual] 検索中...
[text-embedding-granite-embedding-107m-multilingual] 回答生成中...

【text-embedding-granite-embedding-107m-multilingual 回答】
伊方発電所3号機の安全対策について、以下の情報がコンテキストから得られます。

1. **追加安全対策の全体像**: 伊方発電所3号機では、追加の安全対策が実施されています。具体的な内容については、四国電力のプレス発表資料に記載されています。

2. **内部火災防護対策**: 中央制御盤の火災影響軽減のため、鋼板製筐体や金属バリア、配線ダクトを使用して延焼しない離隔距離を確保しています。また、中央制御室には異なる種類の火災感知器や高感度の煙検出設備が設置されています。火災が発生した場合は、二酸化炭素消火器を使用して早期の消火を行うことが計画されています。

3. **緊急時対策所電源の強化**: 緊急時に備えて、電源の強化が図られています。

4. **大規模損壊発生時の体制整備**: 大規模損壊が発生した場合の体制整備が行われています。

5. **不正アクセス防止**: 安全保護回路を設け、不正アクセスを防止する対策が取られています。

これらの情報は、伊方発電所3号機の安全性を高めるために実施されている対策の一部です。詳細な内容については、四国電力の公式資料を参照することが推奨されます。

【参照資料ID】: 37600, 38032, 38064, 35497, 37812, 38367, 37759, 38083, 38202, 35653

【検索結果（上位10件）】

--- Rank 1 (Score: 0.9264) ---
ID: 37600
見出し: ?
開催日: ?
テキスト:
・伊方発電所３号機の追加安全対策の全体像（※事業者によるプレス発表資料：ｈｔｔｐ：／／ｗｗｗ．ｙｏｎｄｅｎ．ｃｏ．ｊｐ／ｐｒｅｓｓ／ｒｅ１３０７／ｄａｔａ／ｐｒ００３－０２

In [17]:
for m in model_dirs:
    print("running:", m.name)
    ans, _ = rag_with_model(question.strip(), m.name, top_k=10)
    print("done:", m.name)

running: text-embedding-granite-embedding-107m-multilingual
[text-embedding-granite-embedding-107m-multilingual] 検索中...
[text-embedding-granite-embedding-107m-multilingual] 回答生成中...
done: text-embedding-granite-embedding-107m-multilingual
running: text-embedding-embeddinggemma-300m
[text-embedding-embeddinggemma-300m] 検索中...
[text-embedding-embeddinggemma-300m] 回答生成中...
done: text-embedding-embeddinggemma-300m
running: text-embedding-mxbai-embed-large-v1
[text-embedding-mxbai-embed-large-v1] 検索中...
[text-embedding-mxbai-embed-large-v1] 回答生成中...
done: text-embedding-mxbai-embed-large-v1
running: text-embedding-embeddinggemma-300m-qat
[text-embedding-embeddinggemma-300m-qat] 検索中...
[text-embedding-embeddinggemma-300m-qat] 回答生成中...
done: text-embedding-embeddinggemma-300m-qat
running: text-embedding-qwen3-embedding-0.6b
[text-embedding-qwen3-embedding-0.6b] 検索中...
[text-embedding-qwen3-embedding-0.6b] 回答生成中...
done: text-embedding-qwen3-embedding-0.6b


RAGAS評価

In [18]:
# （任意）.env を確実に読む。すでに読めてるなら不要
# !pip install python-dotenv
from dotenv import load_dotenv
load_dotenv()

from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
)

from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

from datasets import Dataset
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# ===== RAGAS用LLM設定 =====
RAGAS_LLM_MODEL = "gpt-4o"
RAGAS_EMBED_MODEL = "text-embedding-3-large"

ragas_llm = LangchainLLMWrapper(ChatOpenAI(model=RAGAS_LLM_MODEL))
ragas_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model=RAGAS_EMBED_MODEL))

print("RAGAS準備完了")
print("  評価用LLM:", RAGAS_LLM_MODEL)
print("  評価用Embedding:", RAGAS_EMBED_MODEL)


RAGAS準備完了
  評価用LLM: gpt-4o
  評価用Embedding: text-embedding-3-large


/var/folders/lj/yz56pkwd5rngmq1bzprqp3pr0000gn/T/ipykernel_7329/2883610068.py:24: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  ragas_llm = LangchainLLMWrapper(ChatOpenAI(model=RAGAS_LLM_MODEL))
/var/folders/lj/yz56pkwd5rngmq1bzprqp3pr0000gn/T/ipykernel_7329/2883610068.py:25: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  ragas_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model=RAGAS_EMBED_MODEL))


In [19]:
def prepare_ragas_data(question: str, answer: str, results: pd.DataFrame, ground_truth: str | None = None) -> dict:
    """
    RAGAS評価用のデータを準備
    - question: 質問
    - answer: LLM回答（参照ID付きでもOK。中で除去）
    - results: 検索結果 DataFrame（chunk_text列が入っている想定）
    - ground_truth: 正解文（あれば recall系に使える）
    """
    text_col = "chunk_text" if "chunk_text" in results.columns else "zenbun"

    contexts = results[text_col].tolist() if len(results) > 0 else [""]

    # 回答から参照ID部分を除去（あなたの形式に合わせる）
    answer_clean = answer.split("【参照資料ID】")[0].strip()

    data = {
        "question": question,
        "answer": answer_clean,
        "contexts": contexts,
    }
    if ground_truth is not None and str(ground_truth).strip() != "":
        data["ground_truth"] = str(ground_truth)

    return data

def evaluate_with_ragas(
    question: str,
    answer: str,
    results: pd.DataFrame,
    ground_truth: str | None = None,
    include_recall: bool = False,
) -> pd.DataFrame:
    """
    1問分をRAGASで評価して DataFrame で返す
    """
    ragas_data = prepare_ragas_data(question, answer, results, ground_truth)

    dataset_dict = {
        "question": [ragas_data["question"]],
        "answer": [ragas_data["answer"]],
        "contexts": [ragas_data["contexts"]],
    }

    metrics = [faithfulness, answer_relevancy, context_precision]

    # recallを入れるなら ground_truth 必須
    if include_recall and "ground_truth" in ragas_data:
        dataset_dict["ground_truth"] = [ragas_data["ground_truth"]]
        metrics.append(context_recall)

    dataset = Dataset.from_dict(dataset_dict)

    print(f"RAGAS評価を実行中（LLM: {RAGAS_LLM_MODEL}）...")
    result = evaluate(
        dataset,
        metrics=metrics,
        llm=ragas_llm,
        embeddings=ragas_embeddings,
    )

    return result.to_pandas()


def compare_rag_with_ragas_all(
    question: str,
    model_names: list[str] = None,
    top_k: int = TOP_K,
    ground_truth: str = None
):
    """
    index 配下の複数モデルで RAG → RAGAS評価 → 比較
    """
    print("=" * 60)
    print(f"質問: {question}")
    if ground_truth:
        print(f"正解: {ground_truth[:100]}...")
    print("=" * 60)

    # 評価対象モデル（省略時は index 配下全部）
    if model_names is None:
        model_names = [d.name for d in model_dirs]  # model_dirs は既に作ってある想定

    results_dict = {}

    for model_name in model_names:
        print("\n" + "-" * 40)
        print(f"【{model_name}】")

        # RAG実行
        answer, search_results = rag_with_model(question, model_name, top_k)

        print("\n回答:")
        print(answer)

        print("\nRAGAS評価:")
        try:
            eval_df = evaluate_with_ragas(
                question,
                answer,
                search_results,
                ground_truth=ground_truth,
                include_recall=bool(ground_truth)
            )
            display(eval_df)
            results_dict[model_name] = {
                "answer": answer,
                "search_results": search_results,
                "ragas_eval": eval_df
            }
        except Exception as e:
            print(f"評価エラー: {e}")
            results_dict[model_name] = {
                "answer": answer,
                "search_results": search_results,
                "ragas_eval": None
            }

    # ===== 比較サマリー =====
    print("\n" + "=" * 60)
    print("【評価スコア比較】")
    print("=" * 60)

    comparison_data = []
    for model_name, data in results_dict.items():
        if data["ragas_eval"] is None:
            continue

        row = {"model": model_name}
        for col in data["ragas_eval"].columns:
            if col not in ["question", "answer", "contexts", "ground_truth"]:
                row[col] = data["ragas_eval"][col].values[0]
        comparison_data.append(row)

    if comparison_data:
        comparison_df = pd.DataFrame(comparison_data)
        display(comparison_df)

    return results_dict


In [34]:
# RAGAS評価付きで比較実行（正解あり - Context Recall評価も実施）
question_with_gt = '''
「第100回原子力発電所の新規制基準適合性に係る審査会合」会議はどのような内容でしたか？
'''

ground_truth = '''
原子力発電所 審査会合（第100回）の概要
平成26年3月27日（木）に、原子力規制委員会において「原子力発電所の新規制基準適合性に係る審査会合（第100回）」が開催されました。

会議の基本情報
開催日: 平成26年3月27日（木） 10:00～12:00

場所: 原子力規制委員会 13f 会議室a

目的: 2013年に施行された新規制基準に対する適合性審査。

議題の焦点
主要な議題は、九州電力（株）川内原子力発電所1・2号機の新規制基準への適合性に関する審査で、特に以下の項目が中心となりました。

1. 重大事故対策及び設計基準への適合性
耐震設計の基本方針: 新たに策定された地震動（揺れの大きさ）に対する耐震設計の基本方針について審査されました。（資料1-1）

耐津波設計方針: 基準津波の見直しに伴う施設の評価や、これまでの審査会合で指摘された事項への回答が議論されました。（資料1-2, 1-3, 1-4）

重大事故等対処設備:

事故発生時に使用する可搬型（移動式）の重大事故等対処設備のアクセスルート（運び込む経路）の確保について審査されました。（資料1-5, 1-6）

燃料油貯蔵タンクからこれらの設備へ燃料を輸送する方法に関する指摘事項への回答も議論されました。（資料1-7）
'''

results_with_gt = compare_rag_with_ragas_all(question_with_gt, top_k=5, ground_truth=ground_truth)

質問: 
「第100回原子力発電所の新規制基準適合性に係る審査会合」会議はどのような内容でしたか？

正解: 
原子力発電所 審査会合（第100回）の概要
平成26年3月27日（木）に、原子力規制委員会において「原子力発電所の新規制基準適合性に係る審査会合（第100回）」が開催されました。

会議の基本情報
...

----------------------------------------
【text-embedding-granite-embedding-107m-multilingual】
[text-embedding-granite-embedding-107m-multilingual] 検索中...
[text-embedding-granite-embedding-107m-multilingual] 回答生成中...

回答:
「第100回原子力発電所の新規制基準適合性に係る審査会合」は、平成26年3月27日に開催されました。この会合では、九州電力株式会社の川内原子力発電所1・2号機の重大事故対策及び設計基準への適合性について議論されました。具体的には、地震動に対する耐震設計の基本方針や耐津波設計方針、可搬型重大事故等対処設備のアクセスルートなどが議題に含まれていました。また、これに関連する指摘事項への回答も行われました。資料としては、耐震設計や耐津波設計に関する方針、指摘事項への回答一覧表などが配布されました。

【参照資料ID】: 35563, 35749, 35663, 35693, 37792

RAGAS評価:
RAGAS評価を実行中（LLM: gpt-4o）...


Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.


,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,context_precision,context_recall
0,\n「第100回原子力発電所の新規制基準適合性に係る審査会合」会議はどのような内容でしたか？\n,[原子力発電所の新規制基準適合性に係る審査会合（第１００回）１．日時：平成２６年３月２７日（...,「第100回原子力発電所の新規制基準適合性に係る審査会合」は、平成26年3月27日に開催され...,\n原子力発電所 審査会合（第100回）の概要\n平成26年3月27日（木）に、原子力規制委...,1.0,0.744375,0.833333,0.928571



----------------------------------------
【text-embedding-embeddinggemma-300m】
[text-embedding-embeddinggemma-300m] 検索中...
[text-embedding-embeddinggemma-300m] 回答生成中...

回答:
「第100回原子力発電所の新規制基準適合性に係る審査会合」についての情報は、提供されたコンテキストには含まれていません。したがって、その会議の内容については資料からは読み取れません。

【参照資料ID】: 35749, 35668, 35693, 36079, 35689

RAGAS評価:
RAGAS評価を実行中（LLM: gpt-4o）...


...

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.


,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,context_precision,context_recall
0,\n「第100回原子力発電所の新規制基準適合性に係る審査会合」会議はどのような内容でしたか？\n,[原子力発電所の新規制基準適合性に係る審査会合第１０２回議事録１．日時平成２６年４月３日（木...,「第100回原子力発電所の新規制基準適合性に係る審査会合」についての具体的な内容は、提供され...,\n原子力発電所 審査会合（第100回）の概要\n平成26年3月27日（木）に、原子力規制委...,1.0,0.0,0.0,0.0



【評価スコア比較】


,model,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,context_precision,context_recall
0,text-embedding-granite-embedding-107m-multilin...,\n「第100回原子力発電所の新規制基準適合性に係る審査会合」会議はどのような内容でしたか？\n,[原子力発電所の新規制基準適合性に係る審査会合（第１００回）１．日時：平成２６年３月２７日（...,「第100回原子力発電所の新規制基準適合性に係る審査会合」は、平成26年3月27日に開催され...,\n原子力発電所 審査会合（第100回）の概要\n平成26年3月27日（木）に、原子力規制委...,1.000000,0.744375,0.833333,0.928571
1,text-embedding-embeddinggemma-300m,\n「第100回原子力発電所の新規制基準適合性に係る審査会合」会議はどのような内容でしたか？\n,[原子力発電所の新規制基準適合性に係る審査会合（第１１０回）１．日時：平成２６年５月８日（木...,「第100回原子力発電所の新規制基準適合性に係る審査会合」についての情報は、提供されたコンテ...,\n原子力発電所 審査会合（第100回）の概要\n平成26年3月27日（木）に、原子力規制委...,1.000000,0.000000,0.000000,0.000000
2,text-embedding-mxbai-embed-large-v1,\n「第100回原子力発電所の新規制基準適合性に係る審査会合」会議はどのような内容でしたか？\n,[原子力発電所の新規制基準適合性に係る審査会合（第１５３回）１．日時平成２６年１０月２９日（...,「第100回原子力発電所の新規制基準適合性に係る審査会合」についての具体的な内容は、提供され...,\n原子力発電所 審査会合（第100回）の概要\n平成26年3月27日（木）に、原子力規制委...,1.000000,0.000000,0.000000,0.000000
3,text-embedding-embeddinggemma-300m-qat,\n「第100回原子力発電所の新規制基準適合性に係る審査会合」会議はどのような内容でしたか？\n,[原子力発電所の新規制基準適合性に係る審査会合（第１０１回）１．日時平成２６年４月２日（水）...,「第100回原子力発電所の新規制基準適合性に係る審査会合」は、平成26年3月27日（木）に開...,\n原子力発電所 審査会合（第100回）の概要\n平成26年3月27日（木）に、原子力規制委...,0.428571,0.683607,0.583333,0.928571
4,text-embedding-qwen3-embedding-0.6b,\n「第100回原子力発電所の新規制基準適合性に係る審査会合」会議はどのような内容でしたか？\n,[原子力発電所の新規制基準適合性に係る審査会合第１０２回議事録１．日時平成２６年４月３日（木...,「第100回原子力発電所の新規制基準適合性に係る審査会合」についての具体的な内容は、提供され...,\n原子力発電所 審査会合（第100回）の概要\n平成26年3月27日（木）に、原子力規制委...,1.000000,0.000000,0.000000,0.000000
